In [6]:
import pandas as pd
import numpy as np

In [8]:
tbond_daiy = pd.read_csv('TBond_curve.csv', index_col="Date")
tbond_daiy.index = pd.to_datetime(tbond_daiy.index)
tbond_daiy = tbond_daiy.drop(['20 Yr', '30 Yr', '1.5 Month'], axis=1)
tbond_daiy = tbond_daiy.interpolate(axis=1)
tbond_daiy = tbond_daiy.dropna(axis=0)

In [11]:
maturities = np.unique([
    int(col.split()[0]) / 12 if "Mo" in col else int(col.split()[0])
    for col in tbond_daiy.columns
])

curve_maturities = np.arange(start=maturities.min(), stop=maturities.max()+maturities.min(), step=maturities.min()).round(4)

# Define parameter bounds for optimization: 
bounds = [(0, 1), (-1, 1), (-1, 1), (0, 2)] # Intercept (β0), Slope (β1), Curvature (β2), and Decay Factor (λ)
x0 = [0.01, 0.01, 0.01, 0.5] # Initial guess for the optimization algorithm
method = 'Nelder-Mead' # Optimization method used for minimization

# Aggregate daily bond yields to weekly frequency using median
yields_weekly = tbond_daiy.resample('W').median()
# Keep only the last observation of the dataset
yields_today = tbond_daiy.iloc[-1].to_frame().T

In [10]:
yields_today

,1 Mo,2 Mo,3 Mo,4 Mo,6 Mo,1 Yr,2 Yr,3 Yr,5 Yr,7 Yr,10 Yr
2025-02-25,4.34,4.37,4.3,4.33,4.28,4.12,4.07,4.08,4.12,4.21,4.3
